In [201]:
import pandas as pd
import re
import spacy
from textblob import TextBlob
from nltk.tokenize import word_tokenize
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords

Loading required models and functions

In [ ]:
stop_words = stopwords.words("english")
ttk = TweetTokenizer()
nlp = spacy.load("en")

In [129]:
#tweet_data = pd.read_csv("twitterdata.csv", usecols = [1,2])
#tweet_data = tweet_data.dropna(axis = 0, subset = ['post_text'])
#tweet_data = tweet_data.reset_index(drop=True)

Regex to extract website links, hashtags and mentions from 'post_text'

In [54]:
links = re.compile(r"https?.+")
mentions = re.compile(r"\B@([^\W]+)")
hashtags = re.compile(r"\B#([^\W]+)")

In [188]:
def sentiment_analyzer(text):
    text_blob = TextBlob(text)
    return text_blob.sentiment[0]

def sentiment_score(text):
    sentiment_result = None
    text = text_cleaner_(text)
    polarity = sentiment_analyzer(text)
    if polarity <= -0.6:
        sentiment_result = "Very Bad"
    if polarity > -0.6 and polarity < -0.1:
        sentiment_result = "Bad"
    if polarity < 0.1 and polarity > -0.1:
        sentiment_result = "Neutral"
    if polarity >= 0.1 and polarity < 0.7:
        sentiment_result = "Good"
    if polarity >= 0.7:
        sentiment_result = "Very Good"
    return polarity, sentiment_result

Extract Hashtags, Mentions and url from the text. 
TO DO: Can be scaled to extract more features such as Noun Phrases, adjectives etc using POS tagged data.

In [ ]:
def variable_extractor(df, variable = ['mention', 'hashtag', 'link']):
    if 'link' in variable:
        for row in range(0, len(df)):
           df['text_links'][row] = links.findall(df['post_text'][row]) 
    if 'mention' in variable:
        for row in range(0, len(df)):
           df['text_links'][row] = mentions.findall(df['post_text'][row]) 
    if 'hashtag' in variable:
        for row in range(0, len(df)):
           df['text_links'][row] = hashtags.findall(df['post_text'][row]) 
    return df

def variable_extractor_(text, variable = ['mention', 'hashtag', 'link']):
    link, mention, hashtag = "", "", ""
    if 'link' in variable:
        link = links.findall(text)
    if 'mention' in variable:
        mention = mentions.findall(text)
    if 'hashtag' in variable:
        hashtag = hashtags.findall(text)
    return link, mention, hashtag

Cleaner function to clean mentions, hash tags and stop words from the text. 

In [ ]:
def text_cleaner(tweet_data, variable = ['mention', 'hashtag', 'link']):
    if 'link' in variable:
        for row in range(0, len(df)):
            tweet_data['post_text'][row] = links.sub("", tweet_data['post_text'][row])
    if 'mention' in variable:
        for row in range(0, len(df)):
            tweet_data['post_text'][row] = hashtags.sub("", tweet_data['post_text'][row])
    if 'hashtag' in variable:
        for row in range(0, len(df)):
            tweet_data['post_text'][row] = mentions.sub("", tweet_data['post_text'][row])
    return tweet_data
        
def text_cleaner_(text, variable = ['mention', 'hashtag', 'link', 'stopwords']):
    if 'link' in variable:
        text = links.sub("", text)
    if 'mention' in variable:
        text = hashtags.sub("", text)
    if 'hashtag' in variable:
        text = mentions.sub("", text)
    if 'stopwords' in variable:
        text = " ".join([word for word in word_tokenize(text) if word not in stop_words])
    return text    

Extract _"ORG"_ and _"GPE"_ entities from the text.

In [193]:
def spacy_ner(text):
    text_ = capitalize_mentions(text)
    doc = nlp(text_)
    place = [(ents.text, ents.label_) for ents in doc.ents if ents.label_ in ["ORG", "GPE"]]
    return place

Test the implimentation by passing a single text string.

In [196]:
#19, 21
#text = "Milkbarstore, Andpizza join forces on cereal milk-flavored cookies and cream soda"
print("Enter Text to identify NER and SENTIMENT")
text = input()
print("Sentiment Score: ", sentiment_score(text))
print("Place Entities: ", spacy_ner(text))

Enter Text to identify NER and SENTIMENT
Milkbarstore, Andpizza join forces on cereal milk-flavored cookies and cream soda
Milkbarstore , Andpizza join forces cereal milk-flavored cookies cream soda 0.0
Sentiment Score:  (0.0, 'Neutral')
Place Entities:  [('Milkbarstore', 'GPE')]
